In [2]:
!pip install pyspark


     |███████████████████████████████ | 272.9 MB 35.5 MB/s eta 0:00:01

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



     |████████████████████████████████| 199 kB 126.6 MB/s eta 0:00:01
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824024 sha256=97f3dcd63f2924fe6d3e57cc7e5d71e1498ece4572e3873a0e7c77b4ed02e694
  Stored in directory: /Users/jaskaransingh/Library/Caches/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark


In [8]:
from pyspark.sql.functions import when
from pyspark.sql import SparkSession

In [11]:
spark = SparkSession.builder.appName('CDC').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/15 19:04:19 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [12]:
spark

In [17]:
import os
os.getcwd()

'/Users/jaskaransingh/Desktop/Project/CDC'

In [18]:
!ls

20230315-223617880.csv Glue Job.ipynb         Screenshots
20230315-224015065.csv LOAD00000001.csv


In [88]:
full_load_df = spark.read.csv("LOAD00000001.csv")

In [35]:
display(full_load_df)

DataFrame[_c0: string, _c1: string, _c2: string]

In [71]:
full_load_df.count()

101

In [39]:
full_load_df.columns

['_c0', '_c1', '_c2']

In [89]:
full_load_df = full_load_df.withColumnRenamed('_c0','id').withColumnRenamed('_c1','FullName').withColumnRenamed('_c2','City')

In [73]:
full_load_df.columns

['id', 'FullName', 'City']

In [90]:
full_load_df.write.mode('overwrite').csv('./finalOutput/finalFile.csv')

In [91]:
# UPDATE DATAFRAME (UDF)

udf = spark.read.csv('20230315-223617880.csv')

In [92]:
display(udf)

DataFrame[_c0: string, _c1: string, _c2: string, _c3: string]

In [52]:
udf.count()

6

In [93]:
udf = udf.withColumnRenamed('_c0','action').withColumnRenamed('_c1','id').withColumnRenamed('_c2','FullName').withColumnRenamed('_c3','City')

In [94]:
udf = udf.withColumnRenamed('status','action')
display(udf)

DataFrame[action: string, id: string, FullName: string, City: string]

In [95]:
final_df = spark.read.csv('./finalOutput/finalFile.csv')

In [96]:
final_df.columns

['_c0', '_c1', '_c2']

In [97]:
final_df = final_df.withColumnRenamed('_c0','id').withColumnRenamed('_c1','FullName').withColumnRenamed('_c2','City')

In [98]:
udf.collect()

[Row(action='U', id='7', FullName='ABC XYZ', City='Phoenix'),
 Row(action='I', id='130', FullName='Alica Bing', City='New York'),
 Row(action='I', id='131', FullName='Malinda Bing', City='Detroit'),
 Row(action='I', id='132', FullName='Chandler Bing', City='Portland'),
 Row(action='U', id='8', FullName='ABC XYZ', City='Denver'),
 Row(action='D', id='10', FullName='Jack Hicks', City='Houston')]

In [99]:
for row in udf.collect():
    print(list(row))

['U', '7', 'ABC XYZ', 'Phoenix']
['I', '130', 'Alica Bing', 'New York']
['I', '131', 'Malinda Bing', 'Detroit']
['I', '132', 'Chandler Bing', 'Portland']
['U', '8', 'ABC XYZ', 'Denver']
['D', '10', 'Jack Hicks', 'Houston']


In [100]:
for row in udf.collect():
    
    if row['action'] == 'U':
        final_df = final_df.withColumn('FullName', when(final_df['id'] == row['id'], row['FullName']).otherwise(final_df['FullName']))
        final_df = final_df.withColumn('City', when(final_df['id'] == row['id'], row['City']).otherwise(final_df['City']))
    
    if row['action'] == 'I':
        insertedRow = [list(row)[1:]]
        columns = ['id', 'FullName', 'City']
        new_df = spark.createDataFrame(insertedRow, columns)
        final_df = final_df.union(new_df)
    
    if row['action'] == 'D':
        final_df = final_df.filter(final_df.id != row['id'])
    

In [101]:
for row in final_df.collect():
    print(list(row))

['0', 'Herman Zimmerman', 'Oklahoma City']
['1', 'Lisa Ray', 'Columbus']
['2', 'Terrell Reeves', 'Jacksonville']
['3', 'Steve Goodwin', 'Charlotte']
['4', 'Leah Tran', 'Detroit']
['5', 'Wilbert Holmes', 'Washington']
['6', 'Mindy George', 'Los Angeles']
['7', 'ABC XYZ', 'Phoenix']
['8', 'ABC XYZ', 'Denver']
['9', 'Darla Hayes', 'Charlotte']
['11', 'Francis Davidson', 'Austin']
['12', 'Jerome Padilla', 'San Francisco']
['13', 'Mamie Duncan', 'Houston']
['14', 'Julia Cain', 'San Jose']
['15', 'Leslie Klein', 'Seattle']
['16', 'Isaac Bridges', 'Philadelphia']
['17', 'Martin Adkins', 'Chicago']
['18', 'Vincent Perry', 'Detroit']
['19', 'William Porter', 'Fort Worth']
['20', 'Domingo Byrd', 'Charlotte']
['21', 'Leslie Torres', 'San Francisco']
['22', 'Wanda Morrison', 'El Paso']
['23', 'Gwen Caldwell', 'Indianapolis']
['24', 'Tammy Daniels', 'Indianapolis']
['25', 'Claude Gilbert', 'Phoenix']
['26', 'Monique Bowers', 'Seattle']
['27', 'Virginia Phillips', 'San Francisco']
['28', 'Norman Gra

In [102]:
final_df.write.mode('overwrite').csv('./finalOutput/finalFile2.csv')